In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Training a quantum circuit with PyTorch {#state_preparation}
=======================================

::: {.meta}
:property=\"og:description\": Build and optimize a circuit to prepare
arbitrary single-qubit states, including mixed states, with PyTorch and
PennyLane. :property=\"og:image\":
<https://pennylane.ai/qml/_images/NOON.png>
:::

::: {.related}
tutorial\_qubit\_rotation Basic tutorial: qubit rotation pytorch\_noise
PyTorch and noisy devices tutorial\_isingmodel\_PyTorch 3-qubit Ising
model in PyTorch
:::

*Author: Juan Miguel Arrazola --- Posted: 11 October 2019. Last updated:
25 January 2021.*

In this notebook, we build and optimize a circuit to prepare arbitrary
single-qubit states, including mixed states. Along the way, we also show
how to:

1.  Construct compact expressions for circuits composed of many layers.
2.  Succinctly evaluate expectation values of many observables.
3.  Estimate expectation values from repeated measurements, as in real
    hardware.


The most general state of a qubit is represented in terms of a positive
semi-definite density matrix $\rho$ with unit trace. The density matrix
can be uniquely described in terms of its three-dimensional *Bloch
vector* $\vec{a}=(a_x, a_y, a_z)$ as:

$$\rho=\frac{1}{2}(\mathbb{1}+a_x\sigma_x+a_y\sigma_y+a_z\sigma_z),$$

where $\sigma_x, \sigma_y, \sigma_z$ are the Pauli matrices. Any Bloch
vector corresponds to a valid density matrix as long as
$\|\vec{a}\|\leq 1$.

The *purity* of a state is defined as $p=\text{Tr}(\rho^2)$, which for a
qubit is bounded as $1/2\leq p\leq 1$. The state is pure if $p=1$ and
maximally mixed if $p=1/2$. In this example, we select the target state
by choosing a random Bloch vector and renormalizing it to have a
specified purity.

To start, we import PennyLane, NumPy, and PyTorch for the optimization:


In [ ]:
# we generate a three-dimensional random vector by sampling
# each entry from a standard normal distribution


# purity of the target state


# create a random Bloch vector with the specified purity





# array of Pauli matrices (will be useful later)

Unitary operations map pure states to pure states. So how can we prepare
mixed states using unitary circuits? The trick is to introduce
additional qubits and perform a unitary transformation on this larger
system. By \"tracing out\" the ancilla qubits, we can prepare mixed
states in the target register. In this example, we introduce two
additional qubits, which suffices to prepare arbitrary states.

The ansatz circuit is composed of repeated layers, each of which
consists of single-qubit rotations along the $x, y,$ and $z$ axes,
followed by three CNOT gates entangling all qubits. Initial gate
parameters are chosen at random from a normal distribution. Importantly,
when declaring the layer function, we introduce an input parameter $j$,
which allows us to later call each layer individually.


In [ ]:
# number of qubits in the circuit

# number of layers in the circuit


# randomly initialize parameters from a normal distribution



# a layer of the circuit ansatz

Here, we use the `default.qubit` device to perform the optimization, but
this can be changed to any other supported device.


When defining the QNode, we introduce as input a Hermitian operator $A$
that specifies the expectation value being evaluated. This choice later
allows us to easily evaluate several expectation values without having
to define a new QNode each time.

Since we will be optimizing using PyTorch, we configure the QNode to use
the PyTorch interface:


Our goal is to prepare a state with the same Bloch vector as the target
state. Therefore, we define a simple cost function

$$C = \sum_{i=1}^3 \left|a_i-a'_i\right|,$$

where $\vec{a}=(a_1, a_2, a_3)$ is the target vector and
$\vec{a}'=(a'_1, a'_2, a'_3)$ is the vector of the state prepared by the
circuit. Optimization is carried out using the Adam optimizer. Finally,
we compare the Bloch vectors of the target and output state.


In [ ]:
# cost function








# set up the optimizer


# number of steps in the optimization routine


# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way





# optimization begins















# calculate the Bloch vector of the output state




# print results

About the author
================
